In [1]:
#Change directory to data location
import os
path = 'E:\Daily Reanalisis ERA 5 v2'
os.chdir(path)

In [2]:
import warnings
warnings.simplefilter('ignore') #ignores simple warning

In [3]:
#Import package
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
import matplotlib.ticker as tk
import matplotlib.patches as mpatches
from tqdm import tqdm
import seaborn as sns
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

In [4]:
selectst=['96745', '96741', '96747', '96733', '96749']

In [5]:
datast=pd.read_excel('E:\Data stasiun BMKG\A-Database.xlsx', index_col=2)
datast.index = datast.index.astype(str)
datastslc=datast.loc[selectst]

In [6]:
#Read time-event excel data
censevt=pd.read_excel('calculation_output\CENS Event 3 days Running Mean.xlsx')
censevt=censevt.set_index('time_cens')

In [7]:
#Date event CENS CS no CS (1-->CS, 0-->no CS)
n_cens = 2
evt_cens_cs = []
for i in range(n_cens):
    evt_cens = censevt[censevt['cs event'] == i].index
    print("Banyak kejadian CENS {i} = {n} dari {x}".format(i=i, n=len(evt_cens), x=len(censevt)))
    print("Banyak kejadian CENS {i} = {n}%".format(i=i, n=(len(evt_cens)*100/len(censevt.index))))
    evt_cens_cs.append(evt_cens.values)

Banyak kejadian CENS 0 = 48 dari 117
Banyak kejadian CENS 0 = 41.02564102564103%
Banyak kejadian CENS 1 = 69 dari 117
Banyak kejadian CENS 1 = 58.97435897435897%


In [8]:
evt_cens_cs=[evt_cens_cs[1], evt_cens_cs[0]]

In [9]:
chstmeancs=np.empty([len(evt_cens_cs[0])], dtype=float)
chstmeancsstd=np.empty([len(evt_cens_cs[0])], dtype=float)

for evt in tqdm(range(len(evt_cens_cs[0]))):
    chst=np.empty([len(datastslc)])
    anomchst=np.empty([len(datastslc)])
    for i in range(len(datastslc)):
        data=pd.read_csv('E:/Data stasiun BMKG/Reformat daily/' + datastslc.index[i] +'.csv')
        data['Date']= pd.to_datetime(data['Date'], format='%Y-%m-%d')
        data['RR(mm/dy)'] = pd.to_numeric(data['RR(mm/dy)'], errors='coerce')
        data = data.set_index('Date',drop=False)
        data = data.loc['2001-11-01':'2020-03-31']
        tglcens = np.array([evt_cens_cs[0][evt], evt_cens_cs[0][evt]+np.timedelta64(1,'D')])
        chst[i] = data['RR(mm/dy)'].loc[tglcens[np.isin(tglcens,data['Date'])]].max()
        season = ((data.index.month % 12 + 3) // 3).map({1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'})
        klimdjf = data[season == 'DJF']['RR(mm/dy)'].mean()
        anomchst[i] = chst[i] - klimdjf
    chstmeancs[evt]=np.nanmean(chst)
    chstmeancsstd[evt]=np.nanstd(chst)

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:21<00:00,  3.17it/s]


In [10]:
chcs=pd.DataFrame({'tglcens': evt_cens_cs[0],
                   'chmean':chstmeancs,
                   'chstd':chstmeancsstd
                          })
chcs=chcs.set_index('tglcens')

In [11]:
chstmeanncs=np.empty([len(evt_cens_cs[1])], dtype=float)
chstmeanncsstd=np.empty([len(evt_cens_cs[1])], dtype=float)

for evt in tqdm(range(len(evt_cens_cs[1]))):
    chst=np.empty([len(datastslc)])
    anomchst=np.empty([len(datastslc)])
    for i in range(len(datastslc)):
        data=pd.read_csv('E:/Data stasiun BMKG/Reformat daily/' + datastslc.index[i] +'.csv')
        data['Date']= pd.to_datetime(data['Date'], format='%Y-%m-%d')
        data['RR(mm/dy)'] = pd.to_numeric(data['RR(mm/dy)'], errors='coerce')
        data = data.set_index('Date',drop=False)
        data = data.loc['2001-11-01':'2020-03-31']
        tglcens = np.array([evt_cens_cs[1][evt], evt_cens_cs[1][evt]+np.timedelta64(1,'D')])
        chst[i] = data['RR(mm/dy)'].loc[tglcens[np.isin(tglcens,data['Date'])]].max()
        season = ((data.index.month % 12 + 3) // 3).map({1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'})
        klimdjf = data[season == 'DJF']['RR(mm/dy)'].mean()
        anomchst[i] = chst[i] - klimdjf
    chstmeanncs[evt]=np.nanmean(chst)
    chstmeanncsstd[evt]=np.nanstd(chst)

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:15<00:00,  3.18it/s]


In [12]:
chncs=pd.DataFrame({'tglcens': evt_cens_cs[1],
                   'chmean':chstmeanncs,
                   'chstd':chstmeanncsstd
                          })
chncs=chncs.set_index('tglcens')

In [13]:
#Read Banjir excel data
floodevt=pd.read_excel('data\Data Banjir Jakarta.xlsx')
floodevt['tanggal']= pd.to_datetime(floodevt['tanggal'], format='%Y-%m-%d')
floodevt=floodevt.set_index('tanggal')
floodevt = floodevt.groupby(floodevt.index.floor('D')).sum()

In [14]:
floodevtcens=np.empty(2)
tglcsf=[]
tglncsf=[]
tglcsfd=[]
tglncsfd=[]
bacsf=[]
bancsf=[]
krbcs=[]
krbncs=[]
chcsv=[]
chncsv=[]
chcss=[]
chncss=[]
idxcsv=[]
idxncsv=[]
durcsv=[]
durncsv=[]
for i in range(2):
    totalbanjir=0
    korban=0
    for f in tqdm(range(len(evt_cens_cs[i]))):
        datacens=censevt.loc[evt_cens_cs[i][f]]
        rangecens=pd.date_range(datacens['time_start'], datacens['time_start']+np.timedelta64(datacens['duration']-1,'D'))
        for t in rangecens:
            for e in floodevt.index.values:
                e=[e]
                if (t in e) or (t+np.timedelta64(1,'D') in e) or (t+np.timedelta64(2,'D') in e) or (t+np.timedelta64(-1,'D') in e) or (t+np.timedelta64(-2,'D') in e):
                    totalbanjir=totalbanjir+1
                    if i==0:
                        tglcsf.append(evt_cens_cs[i][f])
                        tglcsfd.append(e[0])
                        bacsf.append((evt_cens_cs[i][f]<=e[0])*1)
                        krbcs.append(floodevt.loc[e[0]]['korban_total'])
                        chcsv.append(chcs.loc[evt_cens_cs[i][f]]['chmean'])
                        chcss.append(chcs.loc[evt_cens_cs[i][f]]['chstd'])
                        idxcsv.append(censevt.loc[evt_cens_cs[i][f]]['maxmagnitude'])
                        durcsv.append(censevt.loc[evt_cens_cs[i][f]]['duration'])
                    elif i==1:
                        tglncsf.append(evt_cens_cs[i][f])
                        tglncsfd.append(e[0])
                        bancsf.append((evt_cens_cs[i][f]<=e[0])*1)
                        krbncs.append(floodevt.loc[e[0]]['korban_total'])
                        chncsv.append(chncs.loc[evt_cens_cs[i][f]]['chmean'])
                        chncss.append(chncs.loc[evt_cens_cs[i][f]]['chstd'])
                        idxncsv.append(censevt.loc[evt_cens_cs[i][f]]['maxmagnitude'])
                        durncsv.append(censevt.loc[evt_cens_cs[i][f]]['duration'])
    floodevtcens[i]=totalbanjir

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:03<00:00, 12.39it/s]


In [17]:
fdcs=pd.DataFrame({'tglcens': tglcsf,
                   'magcens': idxcsv,
                   'meanr': chcsv,
                   'stdr' : chcss,
                   'tglfld':tglcsfd,
                   'victim':krbcs,
                   'before/after CENS'  :bacsf,
                          })
fdcs['tffld']=fdcs.duplicated(subset=['tglfld'])
fdcs['tfcens']=fdcs.duplicated(subset=['tglcens'])
fdcstab=fdcs[(fdcs['tffld']==False) | (fdcs['tfcens']==False)]
fdcstab=fdcstab.drop(columns=['tffld', 'tfcens'])

fdncs=pd.DataFrame({'tglcens': tglncsf,
                    'magcens': idxncsv,
                    'meanr': chncsv,
                    'stdr' : chncss,
                    'tglfld':tglncsfd,
                    'victim':krbncs,
                    'before/after CENS'  :bancsf,
                          })
fdncs['tffld']=fdncs.duplicated(subset=['tglfld'])
fdncs['tfcens']=fdncs.duplicated(subset=['tglcens'])
fdncstab=fdncs[(fdncs['tffld']==False) | (fdncs['tfcens']==False)]
fdncstab=fdncstab.drop(columns=['tffld', 'tfcens'])

writer = pd.ExcelWriter('calculation_output\Flood Event Data CENS.xlsx')
fdcstab.to_excel(writer, 'CENS-CS')
fdncstab.to_excel(writer, 'CENS-noCS')
writer.close()